In [1]:
import os
import time
import copy
import optuna

from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models, datasets, transforms

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
print(torch.cuda.is_available())

c:\Users\Admin\anaconda3\envs\nnd-proj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch Version:  2.1.0+cu118
Torchvision Version:  0.16.0+cu118
True


input dir

In [2]:
CURRENT_DIR = os.getcwd()
MAIN_FOLDER = Path(CURRENT_DIR).parent
OUTPUT_FOLDER = os.path.join(MAIN_FOLDER, 'aligned_augmented')  
FOLD_DATA = os.path.join(MAIN_FOLDER, 'fold_data') 

BATCH_SIZE = 32
NUM_CLASSES = 2
INPUT_SIZE = 224

DEVICE = torch.device("cuda")


Data processing

In [3]:
# Data Transforms
def get_data_transforms(input_size):
    normalize = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    resize = transforms.Resize(input_size + 32)

    return {
        'train': transforms.Compose([resize, transforms.ToTensor(), transforms.Normalize(*normalize)]),
        'val': transforms.Compose([resize, transforms.ToTensor(), transforms.Normalize(*normalize)]),
    }

data_transforms = get_data_transforms(INPUT_SIZE)

In [4]:
def load_gender_data(fold_file):
    gender_map = {}
    with open(fold_file, 'r') as file:
        next(file) 
        for line in file:
            parts = line.strip().split('\t')
            if len(parts) < 5:
                continue  
            user_id = parts[0]
            gender = parts[4]
            if gender.lower() not in ["m", "f"]:
                continue  
            gender_map[user_id] = 0 if gender.lower() == "m" else 1
    return gender_map


In [5]:
class AdienceDataset(Dataset):
    def __init__(self, fold_file, data_folder, transform=None):
        self.gender_map = load_gender_data(fold_file) 
        self.data_folder = data_folder
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Iterate through the user_id subfolders for user_id and gender
        for user_id in self.gender_map:
            user_folder = os.path.join(data_folder, user_id)
            if os.path.isdir(user_folder):
                for img_name in os.listdir(user_folder):
                    if img_name.endswith('.jpg'): 
                        img_path = os.path.join(user_folder, img_name)
                        self.image_paths.append(img_path)
                        self.labels.append(self.gender_map[user_id]) 

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, label

transformers

In [6]:
def get_dataloaders(batch_size, fold_file):
    train_dataset = AdienceDataset(fold_file, OUTPUT_FOLDER, transform=data_transforms['train'])
    val_dataset = AdienceDataset(fold_file, OUTPUT_FOLDER, transform=data_transforms['val'])
    
    print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

    if len(train_dataset) == 0 or len(val_dataset) == 0:
        return None 
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return {'train': train_loader, 'val': val_loader}


Train

In [7]:
# Load Model
def load_model(num_classes, input_size):
    model = models.efficientnet_v2_m(weights="EfficientNet_V2_M_Weights.DEFAULT")
    model.classifier[1] = nn.Linear(1280, num_classes)
    model.num_classes = num_classes
    return model.to(DEVICE)

# Progressively freeze layers
def freeze_layers(model, block_no=7):
    """Freeze layers up to the given block number"""
    for param in model.parameters():
        param.requires_grad = False

    child = list(model.children())
    for i in range(block_no, 9):
        for param in child[0][i].parameters():
            param.requires_grad = True
    return model

In [10]:
def train_model(model, dataloaders, optimizer, num_epochs=25, patience=5, trial=None):
    criterion = nn.CrossEntropyLoss()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epochs_no_improve = 0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:  
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0
            all_preds = []
            all_labels = []

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            acc = accuracy_score(all_labels, all_preds)
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())             

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print("Early stopping")
            break

    model.load_state_dict(best_model_wts)
    print(f'Training complete with Best Val Acc: {best_acc:.4f}')
    return model, history

In [11]:
# Objective Function for Optuna
def objective(trial):
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    block_no = trial.suggest_int("block_no", 4, 7)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])

    best_val_acc = 0.0
    for fold_idx in range(0, 5):
        fold_file = os.path.join(FOLD_DATA, f"fold_{fold_idx}_data.txt")
        print(f"Loading fold file: {fold_file}")
        dataloaders = get_dataloaders(batch_size, fold_file)
        if dataloaders is None:
            print(f"Skipping fold {fold_idx} due to empty dataset.")
            continue


        model = load_model(NUM_CLASSES, INPUT_SIZE)
        model = freeze_layers(model, block_no)

        params_to_update = [p for p in model.parameters() if p.requires_grad]
        optimizer = optim.Adam(params_to_update, lr=lr, weight_decay=weight_decay)

        model, history = train_model(model, dataloaders, optimizer, num_epochs=10, trial=trial)

        best_val_acc = max(best_val_acc, max(history['val_acc']))

    return best_val_acc  # Return the best validation accuracy

In [12]:
# Hyperparameter Optimization Optuna
study = optuna.create_study(direction="maximize")  
study.optimize(objective, n_trials=20)

# Print best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Val Accuracy: {trial.value:.4f}")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-03-28 23:19:35,188] A new study created in memory with name: no-name-77cd3978-998d-4c75-b6ba-8db676f818af


Loading fold file: c:\Users\Admin\Documents\GitHub\CZ4042-SC4001-NND\fold_data\fold_0_data.txt
Train size: 22410, Val size: 22410
Epoch 1/10
----------


In [ ]:
# # Save the model
# torch.save(model, 'efficientnet_test.pth')
# print("Model saved as efficientnet_test.pth")